### A plan how the essay scoring experiment will be done

In [ ]:
'''
1- using the tfIDF vectroizer or the sbert pretrained model to vectroize the sentences into vectors and get the features vectors.

2- then apped the clusters vectors an array of 19 elements to the end of the vectors array.


3- train the model using of the shallow machine learning model.......

'''

In [118]:
from keras.models import Sequential
from keras import layers
from keras_preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import pandas as pd
from sklearn import datasets, svm, tree, metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
#from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import sys
import numpy as np
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import nltk
import re
import matplotlib.pyplot as pltb

In [ ]:
# text= ['hello my name is ahmed and I am a data scientist','data science is very important field','data science is not that hard']
# text1 = ['you are watching unfold data science']
# vectorizer = CountVectorizer()
#
# v = vectorizer.fit(text)
# xv = vectorizer.fit_transform(text)
# print(xv.toarray())
# print(xv.shape)
# vectorizer.get_feature_names_out()
# cv = TfidfVectorizer(min_df=1, stop_words='english')
# xv = cv.fit_transform(text)
# cv.get_feature_names_out()
# print(xv.toarray())

### Building the right file names and reading the files

In [92]:
df = pd.read_csv(r"D:\UDE\6th Semester\MEMS\MEWS Data\TRACE_Datensatz_transposed_220308.csv", sep='\t')
y = df['Code1_IN_AR_Gesamteindruck_Argumentation_3_4_0']  # y is the gold standards

listEssayNames = df['DocumentID']

listEssayNames = [i.removeprefix("00") for i in listEssayNames]

listEssayNames = [i.replace("al","aI") for i in listEssayNames]

listRightNames = []
for essay in listEssayNames:
    x = essay.split("X")
    temp = x[0]+"_"+x[1]+".txt"
    listRightNames.append(temp)
    print(x)

from pathlib import Path

essays =[]
for essayRigt in listRightNames:
    my_file = Path(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\ALLEssaysAllPrompts", essayRigt)
    if my_file.is_file():
        essays.append(my_file.read_text(encoding='utf-8-sig'))

['1017', 'aID2TE']
['1076', 'aID1AD']
['1239', 'aID2TE']
['1326', 'aID1AD']
['1341', 'aID2TE']
['1342', 'aID1AD']
['1056', 'aID2TE']
['1157', 'aID2TE']
['1031', 'aID1AD']
['1040', 'aID1AD']
['1064', 'aID1AD']
['1103', 'aID2TE']
['1161', 'aID1AD']
['1244', 'aID1AD']
['1119', 'aID1AD']
['1159', 'aID2TE']
['1268', 'aID2TE']
['1277', 'aID1AD']
['1063', 'aID1AD']
['1079', 'aID2TE']
['1130', 'aID1AD']
['1158', 'aID1AD']
['1108', 'aID2TE']
['1151', 'aID2TE']
['1178', 'aID1AD']
['1179', 'aID1AD']
['1171', 'aID2TE']
['1243', 'aID1AD']
['1316', 'aID2TE']
['1329', 'aID2TE']
['1306', 'aID2TE']
['1411', 'aID1AD']
['1002', 'aID1AD']
['1004', 'aID2TE']
['1037', 'aID2TE']
['1101', 'aID2TE']
['1162', 'aID1AD']
['1165', 'aID1AD']
['1419', 'aID1AD']
['1442', 'aID2TE']
['1018', 'aID2TE']
['1115', 'aID1AD']
['1479', 'aID2TE']
['1497', 'aID2TE']
['1133', 'aID1AD']
['1251', 'aID1AD']
['1185', 'aID2TE']
['1195', 'aID1AD']
['1264', 'aID1AD']
['1290', 'aID1AD']
['1253', 'aID1AD']
['1259', 'aID1AD']
['1434', 'aI

### CountVectorizer Test

In [ ]:
essays_train, essays_test, y_train, y_test = train_test_split(essays, y, test_size=0.25, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit_transform(essays_train)
X_train = vectorizer.transform(essays_train)
X_test  = vectorizer.transform(essays_test)

print(X_train.shape)
print(X_test.shape)

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)
print("Accuracy:", score)

print(confusion_matrix(y_test, pred))

### Sbert Test

In [ ]:
essays_train, essays_test, y_train, y_test = train_test_split(essays, y, test_size=0.25, random_state=1000)

model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

X_train = model.encode(essays_train, show_progress_bar =True, device="cuda")

X_test  = model.encode(essays_test, show_progress_bar =True, device="cuda")

print(X_train.shape)
print(X_test.shape)

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)
print("Accuracy:", score)

print(confusion_matrix(y_test, pred))